In [1]:
# Librerias, archivos y configuración

import warnings
import os
import polars as pl
import pyarrow
import pandas as pd
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [2]:
# Primero se unen todos los archivos de los distintos meses en uno solo

# Se hace una lista con los archivos de los distintos meses para poder llamar a la función

archivos = os.listdir('../../../../../../../../Volumes/JFreire/Databases/trafic_mad')
archivos.sort(reverse=True)

lista_meses = [e for e in archivos]

In [ ]:
# Se importa y se realiza la limpieza para cada data frame

dfs = []

for e in tqdm(lista_meses):

    # Se importa el archivo

    df = pl.read_csv(f'../../../../../../../../Volumes/JFreire/Databases/trafic_mad/{e}', sep=';', infer_schema_length=0)

    # Realmente solo me interesan las columnas id, fecha, y intensidad. Se borra el resto

    df = df.drop(['id', 'tipo_elem', 'ocupacion', 'carga', 'vmed', 'error', 'periodo_integracion'])

    # Se pasa a pandas para poder agrupar el df por su fecha y hacer la media de la intesidad
    # Lo que se quiere es tener la media de intensidad del tráifco en toda la ciudad

    df = df.to_pandas()

    df['intensidad'] = df.intensidad.astype('int')

    df = df.groupby(['fecha']).mean()

    df.reset_index(inplace=True)

    # Se vuelve a pasar a polars

    df = pl.from_pandas(df)

    # Se añade el df a la lista para su posterior concatenación

    dfs.append(df)

In [ ]:
# Se concatenan todos los data frames en uno solo

df = pl.concat(dfs_2)

In [ ]:
# Se guarda el df en un csv

df.write_csv(file='../final_data/trafic_data.csv')